[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Пример: кластеризация текстов

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Выборка

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train_all = fetch_20newsgroups(subset='train')
print('Список тем', train_all.target_names)

Список тем ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Выберем подмножество из трех тем, сильно отличающихся друг от друга

In [ ]:
simple_dataset = fetch_20newsgroups(
    subset='train', 
    categories=['comp.sys.mac.hardware', 'soc.religion.christian', 'rec.sport.hockey'])

In [ ]:
print(simple_dataset.data[0])

From: erik@cheshire.oxy.edu (Erik Adams)
Subject: HELP!!  My Macintosh "luggable" has lines on its screen!
Organization: Occidental College, Los Angeles, CA 90041 USA.
Distribution: comp
Lines: 20

Okay, I don't use it very much, but I would like for it to keep working
correctly, at least as long as Apple continues to make System software
that will run on it, if slowly :-)

Here is the problem:  When the screen is tilted too far back, vertical
lines appear on the screen.  They are every 10 pixels or so, and seem
to be affected somewhat by opening windows and pulling down menus.
It looks to a semi-technical person like there is a loose connection
between the screen and the rest of the computer.

I am open to suggestions that do not involve buying a new computer,
or taking this one to the shop.  I would also like to not have
to buy one of Larry Pina's books.  I like Larry, but I'm not sure
I feel strongly enough about the computer to buy a service manual
for it.

On a related note:  what

In [ ]:
print('Классы', simple_dataset.target)
print('Количество объектов в выборке', len(simple_dataset.data))

Классы [0 0 1 ... 0 1 2]
Количество объектов в выборке 1777


## Признаки

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=500, min_df=10)
matrix = vectorizer.fit_transform(simple_dataset.data)

In [ ]:
print(matrix.shape)

(1777, 3767)


## Аггломеративная кластеризация (neighbour joining)

В качестве меры близости будем использовать косинусное расстояние между объектами

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=3, affinity='cosine', linkage='complete')
preds = model.fit_predict(matrix.toarray())

Оценим точность кластеризации

In [ ]:
print(float(sum(preds != simple_dataset.target)) / len(simple_dataset.target))

0.674732695554305


Результат плохой, попробуем другой алгоритм

## KMeans

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3, random_state=1)
preds = model.fit_predict(matrix.toarray())
print(preds)

[0 0 2 ... 0 2 1]


In [ ]:
print(simple_dataset.target)

[0 0 1 ... 0 1 2]


По результатам можно предположить, что мы перепутали метки кластеров 1 и 2 местами. Сделаем корректирующее отображение меток и оценим точность алгоритма.

In [ ]:
mapping = {2 : 1, 1: 2, 0: 0}
mapped_preds = [mapping[pred] for pred in preds]
print('Доля ошибок', float(sum(mapped_preds != simple_dataset.target)) / len(simple_dataset.target))

Доля ошибок 0.04670793472144063


Сравном результат кластеризации с прогнозом логистической регрессии

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
clf = LogisticRegression()
print (cross_val_score(clf, matrix, simple_dataset.target).mean())

0.9859313182465581


## Более сложная выборка

Темы в этой выборке семантически более близки друг к другу

In [ ]:
dataset = fetch_20newsgroups(
    subset='train', 
    categories=['comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'comp.graphics'])

In [ ]:
matrix = vectorizer.fit_transform(dataset.data)
model = KMeans(n_clusters=3, random_state=42)
preds = model.fit_predict(matrix.toarray())
print(preds)
print(dataset.target)

[2 1 0 ... 2 0 2]
[2 1 1 ... 2 0 2]


Судя по результатам, метки алгоритма соответствуют целевым, корректирующее отображение не требуется

In [ ]:
mapping = {2 : 0, 1: 1, 0: 2}
mapped_preds = [mapping[pred] for pred in preds]
print('Доля ошибок', float(sum(preds != dataset.target)) / len(dataset.target))

Доля ошибок 0.23217341699942956


In [ ]:
clf = LogisticRegression()
print(cross_val_score(clf, matrix, dataset.target).mean())

0.9264143264143264


На сложной выборке классификатор существенно превосходит кластеризатор по точности

## SVD + KMeans

Попробуем уменшить число признаков с помощью сингулярного разложения матриц

In [ ]:
from sklearn.decomposition import TruncatedSVD

model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=1000, random_state=123)
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)
print(preds)
print(dataset.target)

[0 2 1 ... 0 1 0]
[2 1 1 ... 2 0 2]


Выполним корректирующее отображение меток и оценм точность

In [ ]:
mapping = {0 : 2, 1: 0, 2: 1}
mapped_preds = [mapping[pred] for pred in preds]
print('Доля ошибок', float(sum(mapped_preds != dataset.target)) / len(dataset.target))

Доля ошибок 0.2065031374786081


Попробуем снизить размерность признакового пространства до 200

In [ ]:
model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=200, random_state=123)
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)
print(preds)
print(dataset.target)

[2 0 1 ... 2 1 2]
[2 1 1 ... 2 0 2]


Корректирующее отображение будем подбирать перебором всех возможных вариантов из условия максимизации точности

In [ ]:
import itertools
def validate_with_mappings(preds, target, dataset):
    permutations = itertools.permutations([0, 1, 2])
    for a, b, c in permutations:
        mapping = {2 : a, 1: b, 0: c}
        mapped_preds = [mapping[pred] for pred in preds]
        print(mapping, float(sum(mapped_preds != target)) / len(target))
        
validate_with_mappings(preds, dataset.target, dataset)

{2: 0, 1: 1, 0: 2} 0.9007415858528237
{2: 0, 1: 2, 0: 1} 0.6742726754135767
{2: 1, 1: 0, 0: 2} 0.7056474614945807
{2: 1, 1: 2, 0: 0} 0.8938961779806047
{2: 2, 1: 0, 0: 1} 0.2053622361665716
{2: 2, 1: 1, 0: 0} 0.6200798630918426


Самая лучшая перестановка дает 20% ошибок. Таким образом, существенное снижение размерности признакового пространства не оказало значимого влияния на качество работы алгоритма. Оценим влияние на точность параметра инициализации псевдогенератора случайных чисел (random_state)

In [ ]:
model = KMeans(n_clusters=3, random_state=42)
svd = TruncatedSVD(n_components=200, random_state=321)
features = svd.fit_transform(matrix)
preds = model.fit_predict(features)
print(preds)
print(dataset.target)
validate_with_mappings(preds, dataset.target, dataset)

[2 1 0 ... 2 0 2]
[2 1 1 ... 2 0 2]
{2: 0, 1: 1, 0: 2} 0.713063320022818
{2: 0, 1: 2, 0: 1} 0.845407872219053
{2: 1, 1: 0, 0: 2} 0.8893325727324587
{2: 1, 1: 2, 0: 0} 0.7005134055904164
{2: 2, 1: 0, 0: 1} 0.5864232743867656
{2: 2, 1: 1, 0: 0} 0.2652595550484883


Здесь доля ошибок уже больше - 26%. Следовательно, результаты прогноза могут варьироваться в зависимости от параметров инициализации псевдогенератора случайных чисел.

## Итоги

1. Получили интерпретируемый результат на обеих выборках
1. Реальность, однако, намного более жестока
1. Попробовали использовать AgglomerativeClustering и KMeans